In [2]:
pip install torch_dct


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


In [5]:
import math
import logging
from functools import partial
from collections import OrderedDict
from einops import rearrange, repeat

import torch
import torch_dct as dct
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import load_pretrained
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model


import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix






#from google.colab import drive

# This will prompt for authorization
#drive.mount('/content/drive')



class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class FreqMlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        b, f, _ = x.shape
        x = dct.dct(x.permute(0, 2, 1)).permute(0, 2, 1).contiguous()
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = dct.idct(x.permute(0, 2, 1)).permute(0, 2, 1).contiguous()
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class MixedBlock(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp1 = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        self.norm3 = norm_layer(dim)
        self.mlp2 = FreqMlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        b, f, c = x.shape
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x1 = x[:, :f//2] + self.drop_path(self.mlp1(self.norm2(x[:, :f//2])))
        x2 = x[:, f//2:] + self.drop_path(self.mlp2(self.norm3(x[:, f//2:])))
        return torch.cat((x1, x2), dim=1)


class PoseTransformerV2(nn.Module):
    def __init__(self, num_frame=31, num_joints=20, in_chans=3, embed_dim_ratio=32, depth=4, num_frame_kept=27, num_coeff_kept=27,
                 num_heads=8, mlp_ratio=2., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.2,  norm_layer=None, num_classes = 0):
        """    ##########hybrid_backbone=None, representation_size=None,
        Args:
            num_frame (int, tuple): input frame number
            num_joints (int, tuple): joints number
            in_chans (int): number of input channels, 2D joints have 2 channels: (x,y)
            embed_dim_ratio (int): embedding dimension ratio
            depth (int): depth of transformer
            num_heads (int): number of attention heads
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            qk_scale (float): override default qk scale of head_dim ** -0.5 if set
            drop_rate (float): dropout rate
            attn_drop_rate (float): attention dropout rate
            drop_path_rate (float): stochastic depth rate
            norm_layer: (nn.Module): normalization layer
        """
        super().__init__()

        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        embed_dim = embed_dim_ratio * num_joints   #### temporal embed_dim is num_joints * spatial embedding dim ratio
        out_dim = num_classes    #### output dimension is num_joints * 3

        self.num_frame_kept = num_frame_kept
        self.num_coeff_kept = num_coeff_kept



        ### spatial patch embedding
        self.Joint_embedding = nn.Linear(in_chans, embed_dim_ratio)
        self.Freq_embedding = nn.Linear(in_chans*num_joints, embed_dim)

        self.Spatial_pos_embed = nn.Parameter(torch.zeros(1, num_joints, embed_dim_ratio))
        self.Temporal_pos_embed = nn.Parameter(torch.zeros(1, self.num_frame_kept, embed_dim))
        self.Temporal_pos_embed_ = nn.Parameter(torch.zeros(1, self.num_coeff_kept, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule

        self.Spatial_blocks = nn.ModuleList([
            Block(
                dim=embed_dim_ratio, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])

        self.blocks = nn.ModuleList([
            MixedBlock(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])

        self.Spatial_norm = norm_layer(embed_dim_ratio)
        self.Temporal_norm = norm_layer(embed_dim)

        ####### A easy way to implement weighted mean
        self.weighted_mean = torch.nn.Conv1d(in_channels=self.num_coeff_kept, out_channels=1, kernel_size=1)
        self.weighted_mean_ = torch.nn.Conv1d(in_channels=self.num_frame_kept, out_channels=1, kernel_size=1)

        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim*2),
            nn.Linear(embed_dim*2, out_dim),
        )

    def Spatial_forward_features(self, x):
        b, f, p, _ = x.shape  ##### b is batch size, f is number of frames, p is number of joints
        num_frame_kept = self.num_frame_kept

        index = torch.arange((f-1)//2-num_frame_kept//2, (f-1)//2+num_frame_kept//2+1)

        x = self.Joint_embedding(x[:, index].view(b*num_frame_kept, p, -1))


        x += self.Spatial_pos_embed
        x = self.pos_drop(x)

        for blk in self.Spatial_blocks:
            x = blk(x)

        x = self.Spatial_norm(x)
        x = rearrange(x, '(b f) p c -> b f (p c)', f=num_frame_kept)
        return x

    def forward_features(self, x, Spatial_feature):
        b, f, p, _ = x.shape
        num_coeff_kept = self.num_coeff_kept

        x = dct.dct(x.permute(0, 2, 3, 1))[:, :, :, :num_coeff_kept]
        x = x.permute(0, 3, 1, 2).contiguous().view(b, num_coeff_kept, -1)
        x = self.Freq_embedding(x)

        Spatial_feature += self.Temporal_pos_embed
        x += self.Temporal_pos_embed_
        x = torch.cat((x, Spatial_feature), dim=1)

        for blk in self.blocks:
            x = blk(x)

        x = self.Temporal_norm(x)
        return x

    def forward(self, x):
        b, f, p, _ = x.shape
        x_ = x.clone()

        Spatial_feature = self.Spatial_forward_features(x)
        x = self.forward_features(x_, Spatial_feature)
        x = torch.cat((self.weighted_mean(x[:, :self.num_coeff_kept]), self.weighted_mean_(x[:, self.num_coeff_kept:])), dim=-1)

        x = self.head(x)

        x = x.view(b, num_classes)

        return x


class MyCustomDataset(Dataset):
    def __init__(self, root_dir, mode='train', sliding_window_size=31):
        self.root_dir = root_dir
        self.subjects = os.listdir(root_dir)
        self.frame_data = []  # Stores the information of each frame
        self.labels = [] # Stores the labels of each frame
        self.indexvalues = []
        self.joint_coordinates = []
        self.sliding_window_size = sliding_window_size
        self._process_data(mode)

    def __len__(self):
        return len(self.frame_data)
    


    def __getitem__(self, index):
        #print("index")
        #print(index)
        center_frame = self.frame_data[index]
        #print("center_frame")
        #print(center_frame)
        label = self.labels[index]
        #print("label")
        #print(label)
        skeleton_number = center_frame['skeleton_number']

        #print("skeleton_number")
        #print(skeleton_number)


        skeleton_start_index = self.indexvalues[skeleton_number]['start_index']
        skeleton_end_index = self.indexvalues[skeleton_number]['end_index']

        #print("skeleton_start_index")
        #print(skeleton_start_index)

        #print("skeleton_end_index")
        #print(skeleton_end_index)

        start_index = index - self.sliding_window_size // 2
        end_index = index + self.sliding_window_size // 2

        #print("start_index")
        #print(start_index)

        #print("end_index")
        #print(end_index)

        if (start_index < skeleton_start_index):
          start_index = skeleton_start_index

        else:
          start_index = max(start_index, 0)

        if (end_index > skeleton_end_index):
          end_index = skeleton_end_index



        #print("start_index")
        #print(start_index)

        #print("end_index")
        #print(end_index)

        t = []

        for i in range(start_index, end_index + 1):
          frame_data = self.frame_data[i]
          #print(frame_data['joint_coordinates'])
          joint_coordinates = frame_data['joint_coordinates']
          t.append(joint_coordinates)

        #print("t")
        #print(t)
        #print(len(t))
        while len(t) < self.sliding_window_size:
          t.append([0.0] * 60)  # Assuming 25 joints with (x, y, z) coordinates

        if len(t) < 31 and isinstance(t[0], list):  # Check if t contains elements that are lists
            print("Shape of t before tensor conversion:", len(t), len(t[0]))
        #else:
            #print("Shape of t before tensor conversion:", len(t))

        t = torch.tensor(t, dtype=torch.float32)
        t = t.view(self.sliding_window_size, 20, 3)

        #print(input("Hi"))

        return t, label

    def _process_data(self, mode):
        frame_counter = 0  # Counter for sequential frame numbering
        skeleton_temp=0
        indices = []
        counting = 0

        if mode == 'train':
            data_dir = os.path.join(self.root_dir, 'train')
        elif mode == 'test':
            data_dir = os.path.join(self.root_dir, 'test')
        else:
            raise ValueError("Invalid mode. Mode should be 'train' or 'test'.")



        skeleton_folder = os.listdir(data_dir)

        for skeleton_file in skeleton_folder:
          skeleton_dir = os.path.join(data_dir, skeleton_file)


          action_folder = skeleton_dir[-23:-21]
          #print(action_folder)
          #print(input("hi"))
              
                
        

          # Read and process the skeleton.txt file to extract frame-level data
          frames, labels , start , end= self._read_skeleton_file(skeleton_dir, frame_counter, action_folder , skeleton_temp)
          counting = counting +1
          #print(counting)
          indices.append({
          'start_index': start,
          'end_index': end,

          })
          self.frame_data.extend(frames)
          self.labels.extend(labels)

          # Update the frame counter based on the number of frames in the current skeleton file
          frame_counter += len(frames)
          skeleton_temp = skeleton_temp + 1

        self.indexvalues.extend(indices)


    def _read_skeleton_file(self, skeleton_file, frame_counter, action , skeleton_temp):
        frames = []
        labels = []
        start_ind = frame_counter
        temp_index = 0
        
        #print(skeleton_file)
        # Read and parse the skeleton.txt file to extract frame-level data

        with open(skeleton_file, 'r') as f:
          fcounter = 0
          #first_line = f.readline()
          #first_value = int(first_line.strip())
          #print("frames")
          #print(first_value)


          lines = f.readlines()


          line_counter = 0


          #v = lines[line_counter].strip().split()


          while line_counter < (len(lines) - 20):

            v = lines[line_counter].strip().split()

            #if( v[0] == "0"):
            #line_counter = line_counter+1
            #continue

            if skeleton_file == "/kaggle/input/mal-zip/nt60/TrainingData/S007C001P017R002A042.skeleton":
                print(line_counter)
                print(lines[line_counter])
            #line_counter += 3
            


            frame_number = frame_counter + fcounter
            fcounter=fcounter + 1
            coordinates = []
            # Process the next 25 lines
            for i in range(20):
              # Make sure we don't go beyond the total number of lines in the fil

              if line_counter < len(lines):
                if skeleton_file == "/kaggle/input/mal-zip/nt60/TrainingData/S007C001P017R002A042.skeleton":
                    print(line_counter)
                current_line = lines[line_counter].strip().split()

                # Process the content of the current line here
                for k in current_line[0:3]:
                  #print(k)
                  #print(input("wait"))
                
                  
                  coordinates.append(float(k))
                  #print(coordinates)
                  self.joint_coordinates.append(coordinates)
                  #self.joint_coordinates.append(coordinates)
                  #if skeleton_file == "/kaggle/input/mal-zip/nt60/TrainingData/S007C001P017R002A042.skeleton":
                  #print(coordinates)
                  #print(current_line)  # For example, print the content for demonstration
              line_counter = line_counter + 1

            if skeleton_file == "/kaggle/input/mal-zip/nt60/TrainingData/S007C001P017R002A042.skeleton":
                print("all coordinates")
                print(coordinates)

            # Append the frame data and label to the respective lists
            #print("frame number")
            #print(frame_number)
            #print(coordinates)
            #print(input("wait"))
            frames.append({
            'frame_number': frame_number,
            'joint_coordinates': coordinates,
            'skeleton_number' : skeleton_temp,

            })
            label = action
            labels.append(label)
            # Move to the next block of 3 lines
            #print(self.joint_coordinates)


            if (len(coordinates) != 60):
                print(len(coordinates))
                print("Not correct")
                print(skeleton_file)
                print(input("Hi not correct"))

            if skeleton_file == "/kaggle/input/mal-zip/nt60/TrainingData/S007C001P017R002A042.skeleton":
                print(input("hi end of while"))








        if skeleton_file == "/kaggle/input/mal-zip/nt60/TrainingData/S007C001P017R002A042.skeleton":
            print(input("hi end of file"))

        end_ind = frame_number



        temp_index = frame_number

        #print(coordinates)

        if skeleton_file == "/kaggle/input/mal-zip/nt60/TrainingData/S007C001P017R002A042.skeleton":
            print("start_ind")
            print(start_ind)
            print("temp_index")
            print(temp_index)
            print(input("wait"))


        #print(input("wait"))
        return frames, labels, start_ind, temp_index



print("hey")

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)



# Set the root directory of your dataset
train_root_dir = '/kaggle/input/msr-action-3d/msr action 3d skeleton split'

test_root_dir = '/kaggle/input/msr-action-3d/msr action 3d skeleton split'

#train_root_dir = '/content/drive/My Drive/nt60'

#test_root_dir = '/content/drive/My Drive/nt60'


print("train")
# Create an instance of the training dataset
train_dataset = MyCustomDataset(train_root_dir, mode='train')
print(input("train dataset done"))
print("test")
# Create an instance of the testing dataset
test_dataset = MyCustomDataset(test_root_dir, mode='test')
print(input("test dataset done"))
# Set the batch size
batch_size = 32

# Create data loaders
print("train dataloader")
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(input("train dataloader done"))

print("test dataloader")
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
print(input("test dataloader done"))

train_dataset_length = len(train_dataset)

test_dataset_length = len(test_dataset)

print("train_dataset_length")
print(train_dataset_length)
print("test_dataset_length")
print(test_dataset_length)

hey
cuda
train


train dataset done m


m
test


test dataset done m


m
train dataloader


train dataloader done m


m
test dataloader


test dataloader done m


m
train_dataset_length
13068
test_dataset_length
9162


In [33]:
import random
from sklearn.manifold import TSNE

import matplotlib
matplotlib.use('Agg')  # Use the Agg backend
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import seaborn as sns
from scipy.stats import gaussian_kde
from scipy.optimize import brentq
from scipy.stats import gaussian_kde
from scipy.ndimage import gaussian_filter1d
from matplotlib.lines import Line2D
from matplotlib.patches import Patch


# Set the number of output classes
num_classes = 20 # Number of classes in dataset
emb_dim = num_classes

# Set the hyperparameters
num_frame =31
num_joints = 20
in_chans = 3
embed_dim_ratio = 64
num_frame_kept=27
num_coeff_kept=27
depth = 4
num_heads = 8
mlp_ratio = 2.
qkv_bias = True
qk_scale = None
drop_rate = 0.
attn_drop_rate = 0.
drop_path_rate = 0.2



# Create an instance of the PoseTransformer model
model = PoseTransformerV2(num_frame=num_frame, num_joints=num_joints, in_chans=in_chans, embed_dim_ratio=embed_dim_ratio,
                          num_frame_kept=num_frame_kept,num_coeff_kept=num_coeff_kept,
                        depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                        drop_rate=drop_rate, attn_drop_rate=attn_drop_rate, drop_path_rate=drop_path_rate,
                        num_classes =num_classes)

# Move the model to the device
model = model.to(device)

# Set the optimizer and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Set the loss function
criterion = nn.CrossEntropyLoss()

# Set the number of training epochs
num_epochs = 5

#print(train_dataset.labels)
#print(input("hi"))

# Create a label encoder
label_encoder = LabelEncoder()

# Fit label encoder on the training labels
label_encoder.fit(train_dataset.labels)

# Convert training labels to numerical values
train_labels_encoded = label_encoder.transform(train_dataset.labels)

# Convert the encoded labels to tensors
train_labels_tensor = torch.tensor(train_labels_encoded, dtype=torch.long).to(device)

print("Hi")
print(num_frame)

train_accuracies = []

# Training loop
for epoch in range(num_epochs):
    print(epoch)
    model.train()  # Set the model to training mode
    train_loss = 0.0
    train_correct = 0
    
    all_embeddings_tsne = []

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)

        labels_encoded = label_encoder.transform(labels)  # Convert current batch labels to numerical values
        labels_tensor = torch.tensor(labels_encoded, dtype=torch.long).to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        
        embeddings = outputs
        
        embeddings_tsne = embeddings.view(-1,1,emb_dim)
        
        all_embeddings_tsne.append(embeddings_tsne)

        # Compute the loss
        loss = criterion(outputs, labels_tensor)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Compute the training accuracy
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels_tensor).sum().item()

        # Update the training loss
        train_loss += loss.item() * inputs.size(0)

    # Compute the average training loss and accuracy
    train_loss = train_loss / len(train_dataset)
    train_accuracy = train_correct / len(train_dataset)
    train_accuracies.append(train_accuracy)

    # Print the training loss and accuracy for each epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    
all_embeddings_tsne = torch.cat(all_embeddings_tsne, dim=0)

torch.save(all_embeddings_tsne , '31_5_final_embeddings.pth')

# Save the trained model
torch.save(model.state_dict(), 'trained_msr_action_3d_31_5_pose_transformer_model.pth')



# Testing loop
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_correct = 0
test_predictions = []
test_targets = []
test_lab_pred =[]
true_lab=[]

all_test_embeddings = []

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)

        labels_encoded = label_encoder.transform(labels)  # Convert current batch labels to numerical values
        labels_tensor = torch.tensor(labels_encoded, dtype=torch.long).to(device)


        # Forward pass
        outputs = model(inputs)
        
        embeddings = outputs
        
        embeddings_tsne = embeddings.view(-1,1,emb_dim)
        
        all_test_embeddings.append(embeddings_tsne)

        # Compute the loss
        loss = criterion(outputs, labels_tensor)

        # Compute the testing accuracy
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels_tensor).sum().item()

        # Update the testing loss
        test_loss += loss.item() * inputs.size(0)

        # Collect predictions and targets for computing metrics
        test_predictions.extend(predicted.cpu().numpy())
        test_targets.extend(labels_encoded)
        
        test_lab_pred.append(labels_tensor)
        
        true_lab.append(labels)

    # Compute the average testing loss and accuracy
    test_loss = test_loss / len(test_dataset)
    test_accuracy = test_correct / len(test_dataset)

    # Print the testing loss and accuracy for each epoch
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')




Hi
31
0
Epoch [1/5], Train Loss: 3.0593, Train Accuracy: 0.1472
1
Epoch [2/5], Train Loss: 2.0872, Train Accuracy: 0.3776
2
Epoch [3/5], Train Loss: 1.2036, Train Accuracy: 0.6086
3
Epoch [4/5], Train Loss: 0.9825, Train Accuracy: 0.6735
4
Epoch [5/5], Train Loss: 0.7915, Train Accuracy: 0.7281
Test Loss: 1.3036, Test Accuracy: 0.6359


In [51]:
import matplotlib.pyplot as plt

def plot_accuracy_vs_epochs(train_accuracies, num_epochs):
    plt.figure(figsize=(8, 6))  # Adjust figure size as needed
    plt.plot(range(1, num_epochs + 1), train_accuracies, color='orange', marker='o', linestyle='-')  # Adjust line color as needed
    plt.xlabel('Epochs', fontsize=12)  # Adjust font size as needed
    plt.ylabel('Accuracy', fontsize=12)  # Adjust font size as needed
    plt.title('Training Accuracy vs Epochs', fontsize=14, fontweight='bold')  # Adjust font size and weight as needed
    plt.xticks(fontsize=10)  # Adjust font size of x-axis ticks as needed
    plt.yticks(fontsize=10)  # Adjust font size of y-axis ticks as needed
    #plt.grid(True, linestyle='--', alpha=0.7)  # Add grid lines with custom style and transparency
    plt.gca().set_facecolor('black')  # Set background color to light yellow
    plt.tight_layout()  # Adjust layout to prevent overlap of labels
    plt.savefig('train_accuracy_vs_epochs.png', dpi=300)  # Save plot with high resolution
    plt.close()  # Close the figure to release memory resources

# Example usage:
plot_accuracy_vs_epochs(train_accuracies, num_epochs)


In [45]:
import matplotlib.pyplot as plt

all_embeddings_tsne = torch.cat(all_test_embeddings, dim=0)

embeds = all_embeddings_tsne

targets = torch.cat(test_lab_pred, dim=0)


custom_colors = ['#a26989', '#924ff1', '#6029e8', '#f0215b', '#8da6ae', '#fc8cd9', '#5cc8ff', '#d2cd29', '#fc85a8', '#a55e73', '#89b38f', '#3c2b60', '#27ba26', '#5b274d', '#84421a', '#c142c0', '#97bf08', '#fe978e', '#c3dd15', '#5b7ff6']

# Convert anchor_embeddings to numpy array
embeddings_np = embeds.cpu().numpy()  # Assuming anchor_embeddings is a PyTorch tensor

#Flatten the anchor embeddings to a 2D array
flattened_embeddings = embeds.view(embeds.size(0), -1).cpu().numpy()


# Get the labels for the tsne embeddings
tsne_labels = targets.cpu().numpy()


perplexity_values = [20, 30, 50]
iteration_values = [500, 1000]


plt.figure(figsize=(10, 10))
plot_num = 1
num= 1

cmap = ListedColormap(custom_colors[:len(np.unique(tsne_labels))])

for perplexity in perplexity_values:
    for iterations in iteration_values:
        tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=iterations, random_state=42)
        tsne_embeddings = tsne.fit_transform(flattened_embeddings)
        plt.subplot(len(perplexity_values), len(iteration_values), plot_num)
        plt.scatter(tsne_embeddings[:, 0], tsne_embeddings[:, 1], c=tsne_labels, cmap=cmap , s=1.0)
        plt.title(f'Perplexity: {perplexity}, Iterations: {iterations}')
        plt.show()
        plt.xticks([])
        plt.yticks([])
        plt.savefig('2_tsne_plot' + str(num) + '.png', dpi=300)
        plot_num += 1
        num = num+1
        
plt.close()

In [39]:
import matplotlib.pyplot as plt

# Define custom colors and action names
custom_colors = ['#a26989', '#924ff1', '#6029e8', '#f0215b', '#8da6ae', 
                 '#fc8cd9', '#5cc8ff', '#d2cd29', '#fc85a8', '#a55e73', 
                 '#89b38f', '#3c2b60', '#27ba26', '#5b274d', '#84421a', 
                 '#c142c0', '#97bf08', '#fe978e', '#c3dd15', '#5b7ff6']
action_names = [
    "high arm wave", "horizontal arm wave", "hammer", "hand catch", "forward punch",
    "high throw", "draw x", "draw tick", "draw circle", "hand clap", "two hand wave",
    "side-boxing", "bend", "forward kick", "side kick", "jogging", "tennis swing",
    "tennis serve", "golf swing", "pick up & throw"
]

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 8))

# Plot each color with its corresponding action name using circles
for i, (color, action) in enumerate(zip(custom_colors, action_names)):
    ax.scatter([i]*2, [0, 1], color=color, label=action, marker='o', s=200)  # Use circles as markers

# Hide the axes
ax.axis('off')

# Add legend with adjusted font size
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='large')  # You can adjust font size here

# Save the plot as an image
plt.savefig('color_action_plot.png', bbox_inches='tight')

# Show plot
plt.tight_layout()
plt.show()

plt.close()

In [42]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

action_names = [
    "high arm wave",
    "horizontal arm wave",
    "hammer",
    "hand catch",
    "forward punch",
    "high throw",
    "draw x",
    "draw tick",
    "draw circle",
    "hand clap",
    "two hand wave",
    "side-boxing",
    "bend",
    "forward kick",
    "side kick",
    "jogging",
    "tennis swing",
    "tennis serve",
    "golf swing",
    "pick up & throw"
]

# Assuming you have the label encoder object available
#class_names = label_encoder.classes_

# Print or use the class names as needed
#print(test_lab_pred)
#print(class_names)
#print(input("hi"))
#print(true_lab)
#print(input("hi"))
# Get the predictions and true labels

predictions = np.array(test_predictions)
true_labels = np.array(test_targets)



# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predictions)

# Define custom colormap from black to orange
colors = ["black", "orange"]
cmap = LinearSegmentedColormap.from_list("Custom", colors)

# Compute confusion matrix as percentages
conf_matrix_percentage = conf_matrix / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

# Plot confusion matrix with actual action names
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(conf_matrix_percentage, annot=True, fmt='.1f', cmap=cmap, xticklabels=action_names, yticklabels=action_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.xticks(rotation=90, fontsize=7)  # Adjust font size here
plt.yticks(rotation=0, fontsize=7)  # Adjust font size here
plt.tight_layout()

# Save the figure
plt.savefig('confusion_matrix_percentage.png', dpi=300)

# Show the plot
plt.show()

plt.close()
